In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm = OpenAI(model="gpt-4o")

In [2]:
def add(a,b):
    """ 주어진 두 숫자를 더하고 결과를 출력합니다 """
    return a + b

def mul(a,b):
    """ 주어진 두 숫자를 곱하고 결과를 출력합니다 """
    return a * b

def div(a,b):
    """ 주어진 두 숫자 중 첫 번째 숫자를 두 번째 숫자로 나누고 결과를 출력합니다 """
    return a / b

In [3]:
at = FunctionTool.from_defaults(fn=add)
mt = FunctionTool.from_defaults(fn=mul)
dt = FunctionTool.from_defaults(fn=div)

In [4]:
agent_worker = FunctionCallingAgent.from_tools([at, mt, dt],
                                                    llm=llm,
                                                    verbose=True,
                                                    allow_parallel_tool_calls=False
                                                    )

In [5]:
response = agent_worker.chat("(77*2+2)를 78로 나눈 값을 계산해 줘")
print(response)

> Running step ab7f30ed-5b23-4c07-9a08-b1a6b189f472. Step input: (77*2+2)를 78로 나눈 값을 계산해 줘
Added user message to memory: (77*2+2)를 78로 나눈 값을 계산해 줘
=== Calling Function ===
Calling function: add with args: {"a": 154, "b": 2}
=== Function Output ===
156
> Running step bb1593eb-2d86-40fc-ac83-5204e7c1d6dd. Step input: None
=== Calling Function ===
Calling function: div with args: {"a": 156, "b": 78}
=== Function Output ===
2.0
> Running step 49d827a5-a407-4072-8a3f-f71974a9f76a. Step input: None
=== LLM Response ===
\((77 \times 2 + 2) \div 78\)의 값은 2.0입니다.
\((77 \times 2 + 2) \div 78\)의 값은 2.0입니다.


In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm = OpenAI(model="gpt-4o")

In [7]:
!pip install yfinance==0.2.55
import yfinance as yf


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
def get_stock_price_us(code):
    ticker = yf.Ticker(f"{code}")
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'].iloc[0], 2)

In [9]:
def get_stock_price_Korea(code):
    ticker = yf.Ticker(f"{code}.KS")
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'].iloc[0], 2)

In [10]:
stock_k = FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u = FunctionTool.from_defaults(fn=get_stock_price_us)

In [11]:
agent_worker=FunctionCallingAgent.from_tools([stock_k, stock_u],
                                                llm=llm,
                                                verbose=True,
                                                allow_parallel_tool_calls=False
                                                )

In [12]:
response = agent_worker.chat("TESLA 최신 종가가 어떻게 돼?")

> Running step 582f8e13-64c8-4c38-afe2-47e603d124dc. Step input: TESLA 최신 종가가 어떻게 돼?
Added user message to memory: TESLA 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_us with args: {"code": "TSLA"}
=== Function Output ===
300.71
> Running step e69cb6d3-f160-40c4-9d9c-9b1db14a8347. Step input: None
=== LLM Response ===
TESLA의 최신 종가는 $300.71입니다.


In [13]:
response = agent_worker.chat("삼성전자의 최신 종가가 어떻게 돼?")

> Running step 94009556-cce9-4e55-acb5-bbc99504ccf9. Step input: 삼성전자의 최신 종가가 어떻게 돼?
Added user message to memory: 삼성전자의 최신 종가가 어떻게 돼?
=== Calling Function ===
Calling function: get_stock_price_Korea with args: {"code": "005930"}
=== Function Output ===
60550.0
> Running step 7b03268a-59f2-4a3e-91b5-b9b3716b142c. Step input: None
=== LLM Response ===
삼성전자의 최신 종가는 60,550원입니다.


In [14]:
def get_stock_price_us(code):
    ticker = yf.Ticker(f"{code}")
    todays_data = ticker.history(period='1d')

    if not todays_data.empty:
        close_price = round(todays_data['Close'].iloc[0], 2)
        close_date = todays_data.index[0].strftime('%Y-%m-%d')
        return close_price, close_date

    else:
        return None, None

In [15]:
!pip install curl_cffi # curl_cffi 설치
from curl_cffi import requests

# 미국 주식 최신 종가 호출
def get_stock_price_us(code):
    session = requests.Session(impersonate="chrome")
    ticker = yf.Ticker(f"{code}", session=session)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출
def get_stock_price_Korea(code):
    session = requests.Session(impersonate="chrome")
    ticker = yf.Ticker(f"{code}.KS", session=session)
    todays_data = ticker.history(period='1d')
    return round(todays_data['Close'].iloc[0], 2)

  Using cached curl_cffi-0.11.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (14 kB)
  Using cached cffi-1.17.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached curl_cffi-0.11.4-cp39-abi3-macosx_11_0_arm64.whl (3.0 MB)
Using cached cffi-1.17.1-cp311-cp311-macosx_11_0_arm64.whl (178 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [18]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex
)
from llama_index.core.tools import QueryEngineTool

In [19]:
import os
import urllib.parse
import requests
import re

urls=[
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2022.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2023.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2024.pdf"
]

# 각 파일 다운로드
for url in urls:
    encoded_filename = url.split("/")[-1] # URL에서 파일명 추출
    decoded_filename = urllib.parse.unquote(encoded_filename) # 한글 파일명 복원
    response = requests.get(url)
    if response.status_code == 200:
        # 임시 파일명으로 저장
        temp_filename = "temp_download_file" + os.path.splitext(decoded_filename)[1]
        with open(temp_filename, 'wb') as f:
            f.write(response.content)
        os.rename(temp_filename, decoded_filename) # 파일명 변경
        print(f"완료: {decoded_filename} 다운로드 완료")
    else:
        print(f"오류: {url} 다운로드 실패 (상태 코드: {response.status_code})\n")

완료: [삼성전자]사업보고서_2022.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2023.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2024.pdf 다운로드 완료


In [20]:
S2024_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2024.pdf"]
).load_data()
S2023_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2023.pdf"]
).load_data()
S2022_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2022.pdf"]
).load_data()

In [21]:
S2024_index = VectorStoreIndex.from_documents(S2024_docs)
S2023_index = VectorStoreIndex.from_documents(S2023_docs)
S2022_index = VectorStoreIndex.from_documents(S2022_docs)

In [22]:
S2024_engine = S2024_index.as_query_engine(similarity_top_k=3)
S2023_engine = S2023_index.as_query_engine(similarity_top_k=3)
S2022_engine = S2022_index.as_query_engine(similarity_top_k=3)

In [23]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=S2024_engine,
        name="samsung_2024",
        description=(
            "삼성전자의 2024년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2023_engine,
        name="samsung_2023",
        description=(
            "삼성전자의 2023년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2022_engine,
        name="samsung_2022",
        description=(
            "삼성전자의 2022년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
]

In [24]:
from llama_index.core.agent import FunctionCallingAgent

agent = FunctionCallingAgent.from_tools(tools=query_engine_tools,
                                            llm=llm,
                                            verbose=True,
                                            )

In [25]:
response = agent.chat("2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘")

> Running step d41dbebd-e4ac-4982-8b39-83b783b5eccd. Step input: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
Added user message to memory: 2022년 매출액과 2023년 매출액, 2024년 매출액을 차례로 알려줘
=== Calling Function ===
Calling function: samsung_2022 with args: {"input": "2022\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2022년 삼성전자의 매출액은 302조 2,314억원입니다.
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "2023\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 258조 9,355억원입니다.
=== Calling Function ===
Calling function: samsung_2024 with args: {"input": "2024\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc744 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2024년 삼성전자의 매출액은 300조 8,709억원입니다.
> Running step d1870e2d-7de5-4245-a168-bce4ba077218. Step input: None
=== LLM Response ===
- 2022년 삼성전자의 매출액은 30

In [26]:
response = agent.chat("삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘")

> Running step 60dcad5c-ad04-44a5-92d0-e99d717932bf. Step input: 삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘
Added user message to memory: 삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘
=== Calling Function ===
Calling function: samsung_2023 with args: {"input": "2023\ub144 \uc0bc\uc131\uc804\uc790\uc758 \ub9e4\ucd9c\uc561\uc774 2022\ub144\uc5d0 \ube44\ud574 \uac10\uc18c\ud55c \uc774\uc720\uc640 2024\ub144\uc5d0 \ub2e4\uc2dc \ud68c\ubcf5\ub41c \uc774\uc720\uc5d0 \ub300\ud574 \ubd84\uc11d\ud55c \ub0b4\uc6a9\uc744 \ubcf4\uace0\uc11c\ub97c \ud1b5\ud574 \uc54c\ub824\uc8fc\uc138\uc694."}
=== Function Output ===
2023년 삼성전자의 매출액은 전년 대비 14.3% 감소했습니다. 이는 여러 부문에서의 매출 감소가 원인으로 작용했습니다. 특히 DS 부문에서 32.4%의 큰 감소가 있었고, DX 부문과 SDC 부문도 각각 6.8%와 9.9% 감소했습니다. 반면, Harman 부문은 8.9% 증가했습니다. 이러한 매출 감소는 글로벌 경기 불확실성과 소비심리 악화 등이 영향을 미쳤을 가능성이 있습니다.

2024년에 매출이 회복될 것으로 예상되는 이유는 스마트폰 시장의 경기 회복과 폴더블폰 수요 증가가 긍정적인 영향을 미칠 것으로 보이기 때문입니다